In [ ]:
#| default_exp datasources
%load_ext autoreload
%autoreload 2

import sys,os
from pathlib import Path

In [ ]:
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')

# Fontes de Dados
> Módulo para encapsular a extração e processamento das diferentes fontes de dados

In [ ]:
#| export
import os
import re
from dataclasses import dataclass
from decimal import Decimal, getcontext
from functools import cached_property
from typing import Tuple, Union

import pandas as pd
from dotenv import find_dotenv, load_dotenv
from fastcore.foundation import GetAttr
from fastcore.xtras import Path
from pyarrow import ArrowInvalid, ArrowTypeError

from extracao.connectors import MongoDB, SQLServer
from extracao.constants import (
    AGG_LICENCIAMENTO,
    AGG_SMP,
    BW,
    BW_MAP,
    CHANNELS,
    COLS_LICENCIAMENTO,
    COLUNAS,
    DICT_LICENCIAMENTO,
    DICT_SRD,
    IBGE,
    MONGO_SMP,
    MONGO_SRD,
    MONGO_TELECOM,
    PROJECTION_LICENCIAMENTO,
    PROJECTION_SRD,
    RE_BW,
)

In [ ]:
#| export
getcontext().prec = 5
load_dotenv(find_dotenv())

True

In [ ]:
#| hide: true
#| eval:false
__file__ = Path.cwd().parent / 'extracao' / 'datasources.py'

In [ ]:
#| export

SQLSERVER_PARAMS = dict(
    driver="{ODBC Driver 17 for SQL Server}",
    server="ANATELBDRO05",
    database="SITARWEB",
    trusted_conn=True,
    mult_results=True,
    username=None,
    password=None,
    timeout=1000,
)

MONGO_URI = os.environ.get("MONGO_URI")


@dataclass
class Base:
    folder: Union[str, Path] = Path(__file__).parent / "dados"

    def _read(self, stem: str) -> pd.DataFrame:
        """Lê o dataframe formado por self.folder / self.stem.parquet.gzip"""
        file = Path(f"{self.folder}/{stem}.parquet.gzip")
        try:
            df = pd.read_parquet(file)
        except (ArrowInvalid, FileNotFoundError) as e:
            raise e(f"Error when reading {file}") from e
        return df

    def _save(
        self, df: pd.DataFrame, folder: Union[str, Path], stem: str
    ) -> pd.DataFrame:
        """Format, Save and return a dataframe"""
        df = df.astype("string").drop_duplicates(
            keep="first", ignore_index=True
        )  # TODO: Save this for later
        try:
            file = Path(f"{folder}/{stem}.parquet.gzip")
            df.to_parquet(file, compression="gzip", index=False)
        except (ArrowInvalid, ArrowTypeError) as e:
            raise e(f"Não foi possível salvar o arquivo parquet {file}") from e
        return df

    @cached_property
    def df(self) -> pd.DataFrame:
        try:
            df = self._read(self.stem)
        except (ArrowInvalid, FileNotFoundError):
            df = self._format(self.extract)
        return df

    @cached_property
    def discarded(self) -> pd.DataFrame:
        return pd.DataFrame(columns=self.columns)

    @property
    def columns(self):
        raise NotImplementedError(
            "Subclasses devem implementar a propriedade 'columns'"
        )

    @property
    def cols_mapping(self):
        raise NotImplementedError(
            "Subclasses devem implementar a propriedade 'cols_mapping'"
        )

    @property
    def stem(self):
        raise NotImplementedError("Subclasses devem setar a propriedade stem!")

    def extract(self) -> pd.DataFrame:
        raise NotImplementedError("Subclasses devem implementar o método extract")

    def _format(
        self,
        df: pd.DataFrame,  # DataFrame com os dados de Estações
    ) -> pd.DataFrame:  # DataFrame formatado
        """Formata, limpa e padroniza os dados provenientes da query no banco"""
        raise NotImplementedError("Subclasses devem implementar o método _format")

    def update(self):
        self.df = self._format(self.extract)

    def save(self):
        self._save(self.df, self.folder, self.stem)
        self._save(self.discarded, self.folder, self.stem + "_discarded")


In [ ]:
#| export
class Sitarweb(Base, GetAttr):
    def __init__(self, sql_params: dict = SQLSERVER_PARAMS):
        self.default = SQLServer(sql_params)


In [ ]:
#| export
class Mosaico(Base, GetAttr):
    def __init__(self, mongo_uri: str = MONGO_URI):
        self.database = "sms"
        self.default = MongoDB(mongo_uri)

    @property
    def collection(self):
        raise NotImplementedError(
            "Subclasses devem implementar a propriedade 'collection'"
        )

    @property
    def query(self):
        raise NotImplementedError("Subclasses devem implementar a propriedade 'query'")

    @property
    def projection(self):
        raise NotImplementedError(
            "Subclasses devem implementar a propriedade 'projection'"
        )

    def _extract(self, collection: str, pipeline: list):
        client = self.connect()
        database = client[self.database]
        collection = database[collection]
        result = collection.aggregate(pipeline)
        df = pd.DataFrame(list(result))
        df = df.drop(columns=["_id"])
        return df.astype("string")

    @staticmethod
    def parse_bw(
        bw: str,  # Designação de Emissão (Largura + Classe) codificada como string
    ) -> Tuple[str, str]:  # Largura e Classe de Emissão
        """Parse the bandwidth string"""
        if match := re.match(RE_BW, bw):
            multiplier = BW[match[2]]
            if mantissa := match[3]:
                number = float(f"{match[1]}.{mantissa}")
            else:
                number = float(match[1])
            classe = match[4]
            return str(multiplier * number), str(classe)
        return pd.NA, pd.NA

    @staticmethod
    def split_designacao(
        df: pd.DataFrame,  # DataFrame com coluna original DesignacaoEmissao
    ) -> (
        pd.DataFrame
    ):  # DataFrame com novas colunas Largura_Emissão(kHz) e Classe_Emissão
        """Parse a bandwidth string to extract the numerical component and a character class"""
        df["Designação_Emissão"] = (
            df["Designação_Emissão"].str.replace(",", " ").str.strip().str.upper()
        )
        df["Designação_Emissão"] = df["Designação_Emissão"].str.split(" ")
        df = df.explode("Designação_Emissão")
        df = df[df["Designação_Emissão"] != "/"]  # Removes empty rows
        df[["Largura_Emissão(kHz)", "Classe_Emissão"]] = (
            df["Designação_Emissão"].apply(Mosaico.parse_bw).tolist()
        )
        df[["Largura_Emissão(kHz)", "Classe_Emissão"]] = df[
            ["Largura_Emissão(kHz)", "Classe_Emissão"]
        ].astype("string")
        return df.drop("Designação_Emissão", axis=1)


In [ ]:
#| export
class SRD(Mosaico):
    """Classe para encapsular a lógica de extração de Radiodifusão"""

    def __init__(self, mongo_uri: str = MONGO_URI) -> None:
        super().__init__(mongo_uri)

    @property
    def stem(self):
        return "srd"

    @property
    def collection(self):
        return "srd"

    @property
    def query(self):
        return MONGO_SRD

    @property
    def projection(self):
        return PROJECTION_SRD

    @property
    def columns(self):
        return COLUNAS

    @property
    def cols_mapping(self):
        return DICT_SRD

    @cached_property
    def extract(self) -> pd.DataFrame:
        pipeline = [
            # match the documents that satisfy your query
            {"$match": self.query},
            # project the fields that you want to keep
            {"$project": self.projection},
        ]
        df = self._extract(self.collection, pipeline)
        df.loc[df["estacao"] == "[]", "estacao"] = "{}"
        cols = ["srd_planobasico", "estacao", "habilitacao", "Status"]
        for col in cols:
            df = df.join(pd.json_normalize(df[col].apply(eval)))
        df = df.drop(columns=cols)
        # Substitui strings vazias e somente com espaços por nulo
        return df.replace(r"^\s*$", pd.NA, regex=True)

    def _format(
        self,
        df: pd.DataFrame,  # DataFrame com o resultantes do banco de dados
    ) -> pd.DataFrame:  # DataFrame formatado
        """Formata, limpa e padroniza os dados provenientes da query no banco"""

        df = df.rename(columns=self.cols_mapping)
        df = df[
            df.Status.str.contains("-C1$|-C2$|-C3$|-C4$|-C7|-C98$", na=False)
        ].reset_index(drop=True)
        df["Frequência"] = (
            df.Frequência.astype("string").str.replace(",", ".").astype("float")
        )
        df = df.dropna(subset="Frequência", ignore_index=True)
        df.loc[df["Num_Serviço"] == "205", "Frequência"] = df.loc[
            df["Num_Serviço"] == "205", "Frequência"
        ].apply(lambda x: Decimal(x) / Decimal(1000))
        df["Validade_RF"] = df.Validade_RF.astype("string").str.slice(0, 10)
        df["Fonte"] = "MOSAICO"
        df["Num_Serviço"] = df["Num_Serviço"].fillna("")
        df["Designação_Emissão"] = (
            df.Num_Serviço.astype("string").fillna("").map(BW_MAP)
        )
        df = self.split_designacao(df)
        df["Multiplicidade"] = 1
        return df.loc[:, self.columns]


In [ ]:
#| export
class Telecom(Mosaico):
    """Classe para encapsular a lógica de extração dos serviços de Telecomunições distintos de SMP"""

    def __init__(self, mongo_uri: str = MONGO_URI, limit: int = 0) -> None:
        super().__init__(mongo_uri)
        self.limit = limit

    @property
    def stem(self):
        return "telecom"

    @property
    def collection(self):
        return "licenciamento"

    @property
    def query(self):
        return MONGO_TELECOM

    @property
    def projection(self):
        return PROJECTION_LICENCIAMENTO

    @property
    def columns(self):
        return COLS_LICENCIAMENTO

    @property
    def cols_mapping(self):
        return DICT_LICENCIAMENTO

    @cached_property
    def extract(self) -> pd.DataFrame:
        pipeline = [
            {"$match": self.query},
            {"$project": self.projection},
            {"$limit": self.limit},
        ]
        df = self._extract(self.collection, pipeline)
        # Substitui strings vazias e somente com espaços por nulo
        return df.replace(r"^\s*$", pd.NA, regex=True)

    def _format(
        self,
        df: pd.DataFrame,  # DataFrame com os dados de Estações e Plano_Básico mesclados
    ) -> pd.DataFrame:  # DataFrame com os dados mesclados e limpos
        """Clean the merged dataframe with the data from the MOSAICO page"""
        df = df.rename(columns=self.cols_mapping)
        df = self.split_designacao(df)
        df_copy = df.copy()
        duplicated = df.duplicated(subset=AGG_LICENCIAMENTO, keep="first")
        df_sub = df_copy[~duplicated].reset_index(drop=True)
        df_copy = df_copy[duplicated].reset_index(drop=True)
        df_copy[
            "Log"
        ] = f'[("Registro", {AGG_LICENCIAMENTO},  ("Processamento", "Duplicado considerando os registros")]'

        self.discarded = df_copy
        df_sub["Multiplicidade"] = (
            df.groupby(AGG_LICENCIAMENTO, sort=False).size().tolist()
        )
        df_sub["Status"] = "L"
        df_sub["Fonte"] = "MOSAICO"
        return df_sub.loc[:, self.columns]


In [ ]:
#|eval: false
l = Telecom()

In [ ]:
%%time
#|eval: false
l.update()

CPU times: total: 1min 8s
Wall time: 1min 28s


In [ ]:
#|eval: false
l.df.tail()

,Frequência,Entidade,Fistel,Número_Estação,Município,Código_Município,UF,Latitude,Longitude,Classe,...,Potência(W),Cod_Tipo_Antena,Polarização,Ganho_Antena,FC_Antena,Ang_MP_Antena,Ângulo_Elevação,Azimute,Altura_Antena,Perdas_Acessorias
13230,165.70625,STOCKTOTAL COMERCIO E SERVICOS LTDA,50416688845,1014930453,Valparaiso,3556305,SP,-21.231083,-50.845792,ML,...,40.0,19,V,0.0,0.0,0.0,0.0,0.0,2.0,2.1
13231,392.575,COMUNICATEL ELETRONICA E TELECOMUNICACOES LTDA...,50416256112,1014933614,Goiânia,5208707,GO,-16.661794,-49.254164,BR,...,20.0,35,V,2.1,0.0,0.0,0.0,0.0,20.0,0.0
13232,382.575,COMUNICATEL ELETRONICA E TELECOMUNICACOES LTDA...,50416256112,1014933622,Goiânia,5208707,GO,-16.661794,-49.254164,ML,...,4.0,19,V,0.0,0.0,0.0,0.0,0.0,1.5,0.0
13233,394.0,COMUNICATEL ELETRONICA E TELECOMUNICACOES LTDA...,50416256112,1014933827,Goiânia,5208707,GO,-16.696953,-49.269725,BR,...,20.0,86,V,8.1,0.0,0.0,0.0,0.0,15.0,0.0
13234,384.0,COMUNICATEL ELETRONICA E TELECOMUNICACOES LTDA...,50416256112,1014933835,Goiânia,5208707,GO,-16.696953,-49.269725,ML,...,4.0,19,V,0.0,0.0,0.0,0.0,0.0,1.5,0.0


In [ ]:
#|eval: false
l.discarded.tail()

,Fistel,Num_Serviço,Entidade,UF,Número_Estação,Classe,Latitude,Longitude,Município,Código_Município,...,Altura_Antena,Cod_Tipo_Antena,Polarização,Potência(W),Tipo_Estação,Perdas_Acessorias,Tecnologia,Largura_Emissão(kHz),Classe_Emissão,Log
687015,50416256112,011,COMUNICATEL ELETRONICA E TELECOMUNICACOES LTDA...,GO,1014934050,ML,-16.696953,-49.269725,Goiânia,5208707,...,1.5,19,V,4.0,movelml,0.0,,7.6,F1W,Registro duplicado com base nas colunas: ['Fre...
687016,50416256112,011,COMUNICATEL ELETRONICA E TELECOMUNICACOES LTDA...,GO,1014934068,ML,-16.696953,-49.269725,Goiânia,5208707,...,1.5,19,V,4.0,movelml,0.0,,7.6,F1W,Registro duplicado com base nas colunas: ['Fre...
687017,50416256112,011,COMUNICATEL ELETRONICA E TELECOMUNICACOES LTDA...,GO,1014934076,ML,-16.696953,-49.269725,Goiânia,5208707,...,1.5,19,V,4.0,movelml,0.0,,7.6,F1W,Registro duplicado com base nas colunas: ['Fre...
687018,50416256112,011,COMUNICATEL ELETRONICA E TELECOMUNICACOES LTDA...,GO,1014934084,ML,-16.696953,-49.269725,Goiânia,5208707,...,1.5,19,V,4.0,movelml,0.0,,7.6,F1W,Registro duplicado com base nas colunas: ['Fre...
687019,50416256112,011,COMUNICATEL ELETRONICA E TELECOMUNICACOES LTDA...,GO,1014934092,ML,-16.696953,-49.269725,Goiânia,5208707,...,1.5,19,V,4.0,movelml,0.0,,7.6,F1W,Registro duplicado com base nas colunas: ['Fre...


In [ ]:
#|eval: false
len(l.df) + len(l.discarded)

700255

In [ ]:
l.df.Multiplicidade.sum()

700255

In [ ]:
df = l.split_designacao(l.extract.rename(columns=l.columns))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 700255 entries, 0 to 699999
Data columns (total 28 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Fistel                700255 non-null  string
 1   Num_Serviço           700255 non-null  string
 2   Entidade              700255 non-null  string
 3   UF                    700255 non-null  string
 4   Número_Estação        700255 non-null  string
 5   Classe                700255 non-null  string
 6   Latitude              700255 non-null  string
 7   Longitude             700255 non-null  string
 8   Município             700255 non-null  string
 9   Código_Município      700255 non-null  string
 10  Num_Ato               700255 non-null  string
 11  Validade_RF           700255 non-null  string
 12  Frequência            700255 non-null  string
 13  Frequência_Recepção   700255 non-null  string
 14  Ganho_Antena          700255 non-null  string
 15  FC_Antena             

In [ ]:
#| export
class SMP(Mosaico):
    """Classe para encapsular a lógica de extração do SMP"""

    def __init__(self, mongo_uri: str = MONGO_URI, limit: int = 0) -> None:
        super().__init__(mongo_uri)
        self.limit = limit

    @property
    def stem(self):
        return "smp"

    @property
    def collection(self):
        return "licenciamento"

    @property
    def query(self):
        return MONGO_SMP

    @property
    def projection(self):
        return PROJECTION_LICENCIAMENTO

    @property
    def columns(self):
        return COLS_LICENCIAMENTO

    @property
    def cols_mapping(self):
        return DICT_LICENCIAMENTO

    @cached_property
    def extract(self) -> pd.DataFrame:
        pipeline = [{"$match": self.query}, {"$project": self.projection}]
        if self.limit > 0:
            pipeline.append({"$limit": self.limit})
        df = self._extract(self.collection, pipeline)
        # Substitui strings vazias e somente com espaços por nulo
        return df.replace(r"^\s*$", pd.NA, regex=True)

    def exclude_duplicated(
        self,
        df: pd.DataFrame,  # DataFrame com os dados de Estações
    ) -> pd.DataFrame:  # DataFrame com os dados duplicados excluídos
        f"""Exclui os registros duplicados
        O subconjunto de colunas consideradas é {AGG_SMP}
        """
        df["Número_Estação"] = df["Número_Estação"].astype("int")
        df = df.sort_values("Número_Estação", ignore_index=True)
        df["Largura_Emissão(kHz)"] = pd.to_numeric(
            df["Largura_Emissão(kHz)"], errors="coerce"
        )
        df["Largura_Emissão(kHz)"] = df["Largura_Emissão(kHz)"].fillna(0)
        df["Classe_Emissão"] = df["Classe_Emissão"].fillna("NI")
        df["Tecnologia"] = df["Tecnologia"].fillna("NI")
        duplicated = df.duplicated(subset=AGG_SMP, keep="first")
        df_sub = df[~duplicated].copy().reset_index(drop=True)
        df_sub = df_sub.dropna(subset=AGG_SMP)
        df_sub["Multiplicidade"] = df.groupby(AGG_SMP, sort=False).size().tolist()
        df_sub[
            "Log"
        ] = '[("Registro", "Todos",  ("Processamento", "Duplicado considerando os registros")]'
        df = df[duplicated].reset_index(drop=True)
        df[
            "Log"
        ] = f'[("Registro", {AGG_SMP},  ("Processamento", "Duplicado considerando os registros")]'
        df_sub["Status"] = "L"
        df_sub["Fonte"] = "MOSAICO"
        self.discarded = pd.concat([self.discarded, df], ignore_index=True)
        return df_sub

    @staticmethod
    def read_channels():
        channels = pd.read_csv(CHANNELS, dtype="string")
        channels[["Downlink_Inicial", "Downlink_Final"]] = channels[
            ["Downlink_Inicial", "Downlink_Final"]
        ].astype(float)
        channels[["Uplink_Inicial", "Uplink_Final"]] = channels[
            ["Uplink_Inicial", "Uplink_Final"]
        ].astype(float)
        channels = channels.sort_values(["Downlink_Inicial"], ignore_index=True)
        channels["N_Bloco"] = channels["N_Bloco"].str.strip()
        channels["Faixa"] = channels["Faixa"].str.strip()
        return channels

    def exclude_invalid_channels(self, df):
        df_sub = df[df.Canalização == "Downlink"].copy().reset_index(drop=True)
        for flag in ["Uplink", "Inválida"]:
            discarded = df[df.Canalização == flag]
            if not discarded.empty:
                discarded[
                    "Log"
                ] = f'[("Registro", ("Frequência", "Largura_Emissão(kHz)")),  ("Processamento", "Canalização {flag}")]'
        self.discarded = pd.concat([self.discarded, discarded], ignore_index=True)
        return df_sub

    def validate_channels(
        self,
        df,  # DataFrame with the original channels info
    ) -> pd.DataFrame:  # DataFrame with the channels validated and added info
        """Read the SMP channels file, validate and merge the channels present in df"""
        bw = df["Largura_Emissão(kHz)"].astype("float") / 2000  # Unidade em kHz
        df["Início_Canal_Down"] = df.Frequência.astype(float) - bw
        df["Fim_Canal_Down"] = df.Frequência.astype(float) + bw
        channels = self.read_channels()
        grouped_channels = df.groupby(
            ["Início_Canal_Down", "Fim_Canal_Down"], as_index=False
        ).size()
        grouped_channels.sort_values(
            "size", ascending=False, inplace=True, ignore_index=True
        )
        grouped_channels["Canalização"] = "Inválida"
        for row in grouped_channels.itertuples():
            interval = channels[
                (row.Início_Canal_Down < channels["Downlink_Final"])
                & (row.Fim_Canal_Down > channels["Downlink_Inicial"])
            ]
            faixa = "Downlink"
            if interval.empty:
                interval = channels[
                    (row.Início_Canal_Down < channels["Uplink_Final"])
                    & (row.Fim_Canal_Down > channels["Uplink_Inicial"])
                ]
                if interval.empty:
                    continue
                faixa = "Uplink"

            down = " | ".join(
                interval[["Downlink_Inicial", "Downlink_Final"]].apply(
                    lambda x: f"{x[0]}-{x[1]}", axis=1
                )
            )
            faixas = " | ".join(interval.Faixa.unique())
            offset = " | ".join(interval.Offset.unique())
            grouped_channels.loc[
                row.Index, ["Blocos_Downlink", "Faixas", "Canalização", "Offset"]
            ] = (down, faixas, faixa, offset)
        grouped_channels = grouped_channels[
            [
                "Início_Canal_Down",
                "Fim_Canal_Down",
                "Blocos_Downlink",
                "Faixas",
                "Canalização",
                "Offset",
            ]
        ]
        df = pd.merge(
            df, grouped_channels, how="left", on=["Início_Canal_Down", "Fim_Canal_Down"]
        )
        return self.exclude_invalid_channels(df)

    def generate_uplink(self, df):
        df["Offset"] = pd.to_numeric(df["Offset"], errors="coerce")
        valid = (
            (df.Offset.notna()) & (df.Offset != 0) & (df["Largura_Emissão(kHz)"] != 0)
        )
        df.loc[:, ["Frequência", "Offset"]] = df.loc[
            :, ["Frequência", "Offset"]
        ].astype("float")
        df.loc[valid, "Frequência_Recepção"] = (
            df.loc[valid, "Frequência"] - df.loc[valid, "Offset"]
        ).astype("string")
        return df

    def substitute_coordenates(self, df):
        ibge = pd.read_csv(
            IBGE,
            dtype="string",
            usecols=["codigo_ibge", "nome", "latitude", "longitude"],
        )
        ibge.columns = ["Código_Município", "Município", "Latitude", "Longitude"]
        coords = pd.merge(
            df.loc[df.Multiplicidade != "1", "Código_Município"],
            ibge[["Código_Município", "Latitude", "Longitude"]],
            on="Código_Município",
            how="left",
        )
        df.loc[df.Multiplicidade != "1", ["Latitude", "Longitude"]] = coords[
            ["Latitude", "Longitude"]
        ].values
        df.loc[
            df.Multiplicidade != "1", "Log"
        ] = '[("Registro", ("Latitude", "Longitude")), ("Processamento", "Coordenadas do Município - Registros Agrupados")]'
        return df

    def input_fixed_columns(self, df):
        df["Status"] = "L"
        df["Num_Serviço"] = "010"
        down = df.copy().drop("Frequência_Recepção", axis=1)
        down["Fonte"] = "MOSAICO"
        down["Classe"] = "FB"
        up = df.copy().drop("Frequência", axis=1)
        up = up.rename(columns={"Frequência_Recepção": "Frequência"})
        up["Fonte"] = "CANALIZACAO"
        up["Classe"] = "ML"
        return pd.concat([down, up], ignore_index=True)

    def _format(
        self,
        df: pd.DataFrame,  # DataFrame com os dados de Estações e Plano_Básico mesclados
    ) -> pd.DataFrame:  # DataFrame com os dados mesclados e limpos
        """Clean the merged dataframe with the data from the MOSAICO page"""
        df = df.rename(columns=self.cols_mapping)
        df = self.split_designacao(df)
        df = self.exclude_duplicated(df)
        df = self.validate_channels(df)
        df = self.generate_uplink(df)
        df = self.substitute_coordenates(df)
        df = self.input_fixed_columns(df)
        return df.loc[:, self.columns]


if __name__ == "__main__":
    import time

    start = time.perf_counter()

    l = SMP()

    l.update()

    print(l.df)

    print(150 * "=")

    print(l.discarded[["Frequência", "Entidade", "Log"]])

    print(150 * "=")

    print(l.df.Multiplicidade.sum())

    l.save()

    print(f"Elapsed time: {time.perf_counter() - start} seconds")

In [ ]:
#| hide
from nbdev.doclinks import nbdev_export 
nbdev_export()